In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy as scp

from itertools import product

%matplotlib inline

headers = ['age','workclass','fnlwgt','education','education-num','marital-status',
           'occupation','relationship','race','sex','capital-gain','capital-loss',
           'hours-per-week','native-country','class']
adult = pd.read_csv('./datasets/adult.data', 
                    sep=', ', names=headers, na_values='?', engine='python')

In [40]:
# Drop all records with missing values
adult.dropna(inplace=True)
adult.reset_index(drop=True, inplace=True)

# Drop fnlwgt, not interesting for ML
adult.drop('fnlwgt', axis=1, inplace=True)
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 14 columns):
age               30162 non-null int64
workclass         30162 non-null object
education         30162 non-null object
education-num     30162 non-null int64
marital-status    30162 non-null object
occupation        30162 non-null object
relationship      30162 non-null object
race              30162 non-null object
sex               30162 non-null object
capital-gain      30162 non-null int64
capital-loss      30162 non-null int64
hours-per-week    30162 non-null int64
native-country    30162 non-null object
class             30162 non-null object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [41]:
# Convert objects to categories
# Education is ordered
ordered_education = ['Preschool','1st-4th','5th-6th','7th-8th','9th',
                    '10th','11th','12th','HS-grad','Assoc-acdm',
                     'Assoc-voc','Some-college','Bachelors','Prof-school',
                     'Masters','Doctorate']
adult['education'] = adult['education'].astype(pd.api.types.CategoricalDtype(categories=ordered_education, ordered=True))

#Hours per week is

# The rest are not
obj_columns = adult.select_dtypes(['object']).columns
adult[obj_columns] = adult[obj_columns].astype('category')

In [42]:
# Convert numerics to floats and normalize
num_columns = adult.select_dtypes(['int64']).columns
adult[num_columns] = adult[num_columns].astype('float64')
for c in num_columns:
    #adult[c] -= adult[c].mean()
    #adult[c] /= adult[c].std()
    adult[c] /= (adult[c].max()-adult[c].min())
adult['class'] = adult['class'].cat.codes
display(adult.info())
display(adult.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 14 columns):
age               30162 non-null float64
workclass         30162 non-null category
education         30162 non-null category
education-num     30162 non-null float64
marital-status    30162 non-null category
occupation        30162 non-null category
relationship      30162 non-null category
race              30162 non-null category
sex               30162 non-null category
capital-gain      30162 non-null float64
capital-loss      30162 non-null float64
hours-per-week    30162 non-null float64
native-country    30162 non-null category
class             30162 non-null int8
dtypes: category(8), float64(5), int8(1)
memory usage: 1.4 MB


None

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,0.534247,State-gov,Bachelors,0.866667,Never-married,Adm-clerical,Not-in-family,White,Male,0.021740,0.0,0.408163,United-States,0
1,0.684932,Self-emp-not-inc,Bachelors,0.866667,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.000000,0.0,0.132653,United-States,0
2,0.520548,Private,HS-grad,0.600000,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.000000,0.0,0.408163,United-States,0
3,0.726027,Private,11th,0.466667,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.000000,0.0,0.408163,United-States,0
4,0.383562,Private,Bachelors,0.866667,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.000000,0.0,0.408163,Cuba,0
5,0.506849,Private,Masters,0.933333,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.000000,0.0,0.408163,United-States,0
6,0.671233,Private,9th,0.333333,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.000000,0.0,0.163265,Jamaica,0
7,0.712329,Self-emp-not-inc,HS-grad,0.600000,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.000000,0.0,0.459184,United-States,1
8,0.424658,Private,Masters,0.933333,Never-married,Prof-specialty,Not-in-family,White,Female,0.140841,0.0,0.510204,United-States,1
9,0.575342,Private,Bachelors,0.866667,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.051781,0.0,0.408163,United-States,1


In [43]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


display(adult.head())
enc = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', [0, 3, 9, 10, 11]),
        #('num', 'passthrough', [0, 2, 8, 9, 10]),
        ('cat', OneHotEncoder(), [1, 2, 4, 5, 6, 7, 8, 12]),])
        #('cat', OneHotEncoder(), [1, 3, 4, 5, 6, 7, 11]),])
        #('label', LabelEncoder(), 13)])

enc.fit(adult)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,0.534247,State-gov,Bachelors,0.866667,Never-married,Adm-clerical,Not-in-family,White,Male,0.02174,0.0,0.408163,United-States,0
1,0.684932,Self-emp-not-inc,Bachelors,0.866667,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00000,0.0,0.132653,United-States,0
2,0.520548,Private,HS-grad,0.600000,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.00000,0.0,0.408163,United-States,0
3,0.726027,Private,11th,0.466667,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.00000,0.0,0.408163,United-States,0
4,0.383562,Private,Bachelors,0.866667,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.00000,0.0,0.408163,Cuba,0


ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', 'passthrough', [0, 3, 9, 10, 11]), ('cat', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True), [1, 2, 4, 5, 6, 7, 8, 12])])

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(adult.drop('class', axis=1), adult['class'], test_size=0.33, random_state=42)

In [48]:
# Load semantic distances from files

import os
import collections

#path = './datasets/semdist/ontodistnolog/'
#path = './datasets/semdist/ontodist/'
path = './datasets/semdist/embedist/'
categorical_attributes = list(adult.select_dtypes(['category']).columns.values)
categorical_attributes = categorical_attributes[:-1]

distances = {}
for categorical_attribute in categorical_attributes:
    with open(path+categorical_attribute+'.txt') as f:
        contents = list(f)
        categories = list(map(str.strip, contents[0].split(',')))
        m = []
        for line in contents[1:]:
            m.append(list(map(float, map(str.strip, line.split(',')))))
        d = collections.defaultdict(dict)
        for i in range(len(categories)):
            for j in range(len(categories)):
                d[categories[i]][categories[j]] = m[i][j]
        distances[categorical_attribute] = d

In [49]:
import math

# current best results embedist/weight/nosquare
def dist_attr(x, y, attr_pos, categorical_mask, column_names):
    if categorical_mask[attr_pos]:
        #return 0.625 * distances[column_names[attr_pos]][x[attr_pos]][y[attr_pos]]**2
        #return distances[column_names[attr_pos]][x[attr_pos]][y[attr_pos]]**2
        #return 0.625 * distances[column_names[attr_pos]][x[attr_pos]][y[attr_pos]]
        return distances[column_names[attr_pos]][x[attr_pos]][y[attr_pos]]
    else:
        #return (float(x[attr_pos]) - float(y[attr_pos]))**2
        return abs(float(x[attr_pos]) - float(y[attr_pos]))

def dist_record(x, y):
    d = []
    categorical_mask = [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1]
    #categorical_mask = [0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1]
    column_names = list(adult.columns.values)[:-1]
    for i in range(len(column_names)):
        d.append(dist_attr(x, y, i, categorical_mask, column_names))
    
    #return float(math.sqrt(sum(d)))
    return float(sum(d))
    
def mean_record(D):
    d = []
    # 0->age 
    d.append(float(np.mean(D[:,0])))
    # 1->workclass  
    d.append(mean_semantic(D[:,1], 'workclass'))
    # 2->education 
    d.append(mean_semantic(D[:,2], 'education'))
    # 3->education-num   
    d.append(float(np.mean(D[:,3])))
    # 4->marital-status  
    d.append(mean_semantic(D[:,4], 'marital-status'))
    # 5->occupation  
    d.append(mean_semantic(D[:,5], 'occupation'))
    # 6->relationship 
    d.append(mean_semantic(D[:,6], 'relationship'))
    # 7->race    
    d.append(mean_semantic(D[:,7], 'race'))
    # 8->sex  
    d.append(mean_semantic(D[:,8], 'sex'))
    # 9->capital-gain  
    d.append(float(np.mean(D[:,9])))
    #10->capital-loss 
    d.append(float(np.mean(D[:,10])))
    #11->hours-per-week 
    d.append(float(np.mean(D[:,11])))
    #12->native-country  
    d.append(mean_semantic(D[:,12], 'native-country'))
    
    return d
    
def mean_semantic(values, attribute_name):
    candidates = list(distances[attribute_name].keys())
    return values[np.argmin([sum([distances[attribute_name][c][v] for c in candidates]) for v in values])]

def dist(x,y):
    return np.linalg.norm(x-y)
    #return scipy.spatial.distance.correlation(x,y)

def poprow(arr,i):
    pop = arr[i]
    new_array = np.vstack((arr[:i],arr[i+1:]))
    return new_array,pop

def cluster(X, p, k):
    c = [p]
    D = np.column_stack((X,[dist_record(v[:-1],p[:-1]) for v in X]))
    D = D[D[:,-1].argsort()]
    D = np.delete(D, -1, 1)
    c.extend(D[:k-1])
    D = D[k-1:]
    
    xc = np.array([p[:-1] for p in c], copy=False, ndmin=2)
    yc = np.array([p[-1] for p in c], copy=False)
    cl = (xc, yc)
    return D, cl
    
def mdav(X, y, k):
    D = np.column_stack((X,y))
    clusters = []
    while len(D) >= 3*k:
        # Centroid
        xm = mean_record(D)
        # Furthest from centroid
        xri = np.argmax([dist_record(v[:-1],xm) for v in D])
        D, xr = poprow(D, xri)
        # Furthest from furthest from centroid
        xsi = np.argmax([dist_record(v[:-1],xr[:-1]) for v in D])
        D, xs = poprow(D, xsi) 

        #cluster of xr
        D, c = cluster(D, xr, k)
        clusters.append(c)
        #cluster of xs
        D, c = cluster(D, xs, k)
        clusters.append(c)
        
    if len(D) >= 2*k and len(D) < 3*k:
        # Centroid
        xm = mean_record(D)
        # Furthest from centroid
        xri = np.argmax([dist_record(v[:-1],xm) for v in D])
        D, xr = poprow(D, xri)
        #cluster of xr
        D, c = cluster(D, xr, k)
        clusters.append(c)
        
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        cl = (xc, yc)
        clusters.append(cl)     
    else:
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        cl = (xc, yc)
        clusters.append(cl)
    
    centroids = np.array([mean_record(c[0]) for c in clusters], copy=False)
    
    return clusters, centroids

from sklearn import tree
def gen_explanations(clustering, max_depth=-1):
    explanations = []
    for cluster in clustering:
        # Testing with max depth
        if max_depth < 1:
            exp = tree.DecisionTreeClassifier()
        else:
            exp = tree.DecisionTreeClassifier(max_depth=max_depth)
        exp.fit(enc.transform(cluster[0]),cluster[1])
        explanations.append(exp) 
    return explanations

def pre_explanations(explanations, centroids, X):
    predictions = []
    for sample in X:
        #select the closest classifier
        exp = explanations[np.argmin([dist_record(sample,c) for c in centroids])]
        #exp_pred = exp.predict([sample])
        exp_pred = exp.predict(enc.transform([sample]))
        predictions.append(exp_pred[0])
    return predictions

def pre_explanations_ext(explanations, centroids, X, T, n):
    predictions = []
    for sample, truth in zip(X,T):
        #select the 3 closest classifiers
        mins = np.array([dist_record(sample,c) for c in centroids]).argsort()[:n]
        for m in mins:
            exp = explanations[m]
            #exp_pred = exp.predict([sample])
            exp_pred = exp.predict(enc.transform([sample]))
            if(exp_pred[0] == truth):
                break
        predictions.append(exp_pred[0])
    return predictions

In [51]:
# Generate clusters for different representativities
representativity = [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
K = [int(len(X_train)*r) for r in representativity]
clusterings = []
centroids_of_clusterings = []
for k in K:
    clustering, centroids = mdav(X_train, y_train, k)
    clusterings.append(clustering)
    centroids_of_clusterings.append(centroids)    

KeyError: 'native-country'

In [18]:
# Train explanations
clu_explanations = []
for clustering in clusterings:
    # Test shallow trees depht=4
    explanations = gen_explanations(clustering, 4)
    clu_explanations.append(explanations)

In [19]:
# Train blackbox model
from sklearn.neural_network import MLPClassifier
blackbox = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
blackbox.fit(enc.transform(X_train), y_train)
display(blackbox.score(enc.transform(X_test), y_test))

TypeError: object of type 'CategoricalDtype' has no len()

In [20]:
# Train big tree
surrogate = tree.DecisionTreeClassifier()
surrogate.fit(enc.transform(X_train),y_train)
display(surrogate.score(enc.transform(X_test), y_test))
display(surrogate.tree_.node_count)

TypeError: object of type 'CategoricalDtype' has no len()

In [ ]:
truth = y_test
blackbox_predictions = blackbox.predict(enc.transform(X_test))
surrogate_predictions = surrogate.predict(enc.transform(X_test))
explanation_predictions = []
explanation_ext_predictions = []
for i in range(len(clu_explanations)):
    p = pre_explanations(clu_explanations[i], centroids_of_clusterings[i], np.array(X_test))
    q = pre_explanations_ext(clu_explanations[i], centroids_of_clusterings[i], np.array(X_test), blackbox_predictions, 3)
    explanation_predictions.append(p)
    explanation_ext_predictions.append(q)

In [ ]:
# acc = TP+TN/all
acc_blackbox = np.mean([t==p for t,p in zip(truth, blackbox_predictions)])
acc_surrogate = np.mean([t==p for t,p in zip(truth, surrogate_predictions)])
acc_explanations = []
for i in range(len(explanation_predictions)):
    acc_explanations.append(np.mean([t==p for t,p in zip(truth, explanation_predictions[i])]))
    
acc_explanations_ext = []
for i in range(len(explanation_ext_predictions)):
    acc_explanations_ext.append(np.mean([t==p for t,p in zip(truth, explanation_ext_predictions[i])]))

acc_cross = []
for i in range(len(explanation_predictions)):
    acc_cross.append(np.mean([t==p for t,p in zip(blackbox_predictions, explanation_predictions[i])]))

acc_cross_ext = []
for i in range(len(explanation_predictions)):
    acc_cross_ext.append(np.mean([t==p for t,p in zip(blackbox_predictions, explanation_ext_predictions[i])]))

display(acc_blackbox)
display(acc_explanations)
display(acc_explanations_ext)
display(acc_cross)
display(acc_cross_ext)

In [ ]:
min_nodes = []
max_nodes = []
mean_nodes = []
median_nodes = []
n_counts = []
for explanations in clu_explanations:
    node_counts = [exp.tree_.node_count for exp in explanations]
    n_counts.append(node_counts)
    min_nodes.append(np.min(node_counts))
    max_nodes.append(np.max(node_counts))
    mean_nodes.append(np.mean(node_counts))
    median_nodes.append(np.median(node_counts))

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline

representativity = [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
representativity = list([r*100 for r in representativity])

xnew = np.linspace(min(representativity),max(representativity),300)
spl = make_interp_spline(representativity, acc_explanations, k=3) #BSpline object
ynew = spl(xnew)

spl = make_interp_spline(representativity, acc_explanations_ext, k=3) #BSpline object
ynew4 = spl(xnew)

acc_blackbox_const = [acc_blackbox]*len(representativity)
spl = make_interp_spline(representativity, acc_blackbox_const, k=3) #BSpline object
ynew2 = spl(xnew)

big_tree = [acc_surrogate]*len(representativity)
spl = make_interp_spline(representativity, big_tree, k=3) #BSpline object
ynew6 = spl(xnew)

spl = make_interp_spline(representativity, acc_cross, k=3) #BSpline object
ynew3 = spl(xnew)

spl = make_interp_spline(representativity, acc_cross_ext, k=3) #BSpline object
ynew5 = spl(xnew)

plt.figure(figsize=(9,7))
plt.plot(xnew,ynew2,linestyle='-',linewidth=3,color='k',label='ANN')
plt.plot(xnew,ynew6,linestyle='-.',linewidth=3,color='k',label='Global DT')
plt.plot(xnew,ynew,linestyle=':',linewidth=3,color='k',label='Unguided DT')
#plt.plot(xnew,ynew3, label='interpretable w.r.t. blackbox')
plt.plot(xnew,ynew4,linestyle='--',linewidth=3,color='k',label='Guided DT')

#plt.plot(xnew,ynew5, label='interpretable-guided w.r.t. blackbox')


plt.legend(handlelength=4)
plt.ylim(0.7,0.9)
plt.xlabel('% of records per cluster')
plt.ylabel('classification accuracy')
plt.grid()
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(9,7))
##plt.plot(xnew,ynew2, label='blackbox')
##plt.plot(xnew,ynew, label='interpretable')
plt.plot(xnew,ynew3,linestyle='-',linewidth=3,color='k',label='Unguided DT w.r.t. ANN predictions')
##plt.plot(xnew,ynew4, label='interpretable-guided')
plt.plot(xnew,ynew5,linestyle=':',linewidth=3,color='k',label='Guided DT w.r.t. ANN predictions')

plt.legend()
plt.ylim(0.7,1)
plt.xlabel('% of records per cluster')
plt.ylabel('classification accuracy')
plt.grid()
plt.show()
plt.close()

In [ ]:
plt.plot(xnew,make_interp_spline(representativity, min_nodes, k=3)(xnew),label='min')
plt.plot(xnew,make_interp_spline(representativity, max_nodes, k=3)(xnew),label='max')
plt.plot(xnew,make_interp_spline(representativity, mean_nodes, k=3)(xnew),label='mean')
plt.plot(xnew,make_interp_spline(representativity, median_nodes, k=3)(xnew),label='median')
plt.legend()
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(9,7))
plt.boxplot(n_counts)
locs, _ = plt.xticks()
plt.xticks(locs, representativity)
#plt.yscale('log')
plt.xlabel('% of records per cluster')
plt.ylabel('number of nodes')
#plt.ylabel('number of nodes (log scale)')
plt.grid()
plt.show()
plt.close()

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

for clu_explanation in clu_explanations:
    dot_data = StringIO()
    export_graphviz(clu_explanation[0], out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True)

    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    display(Image(graph.create_png()))

    